In [6]:

# Importación de librerias necesarias para el analisis exploratorio (EDA)
# -------------------------------------------------------------------------------
# Tratamiento de datos
import pandas as pd
import numpy as np
import os #Gestion de archivos y rutas

# Visualizacion de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Estadisticas y evaluacion de relaciones entre variables
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Configuracion de Pandas y eliminacion de warnings
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en los DataFrames
import warnings
warnings.filterwarnings("ignore")  # Ocultar warnings innecesarios

## DEFINIR FUNCIONES DE CARGA Y EXPLORACION ✨

In [7]:
def cargar_datos(ruta):
    """
    Carga un archivo CSV en un DataFrame de Pandas y muestra sus primeras filas.

    Parametros:
        ruta (str): Ruta del archivo CSV a cargar.

    Retorna:
        pd.DataFrame: DataFrame con los datos cargados.
    """
    if not os.path.exists(ruta):
        print(f"⚠ Error: El archivo '{ruta}' no existe.")
        return None  # Retornar None si el archivo no existe
    
    try:
        df = pd.read_csv(ruta)
        print(f"✅ Datos cargados correctamente. Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
        display(df.head())  # Mostrar las primeras filas
        return df
    except Exception as e: # Capturar posibles errores inesperados y guardarlos en la variable e
        print(f"⚠ Error al cargar el archivo: {e}")
        return None 


In [10]:
def revisar_estructura(df):
    """
    Muestra informacion basica sobre la estructura del DataFrame.

    Parametros:
        df (pd.DataFrame): DataFrame a analizar.

    Retorna:
        None
    """
    print("📌 Dimensiones del dataset:")
    print(f"  - Filas: {df.shape[0]}")
    print(f"  - Columnas: {df.shape[1]}\n")

    print("📌 Nombres de las columnas:")
    print(df.columns.tolist(), "\n")

    print("📌 Tipos de datos:")
    print(df.dtypes, "\n")

    # Identificar duplicados
    duplicados = df.duplicated().sum()
    print(f"📌 Datos duplicados: {duplicados} ({round(duplicados / df.shape[0] * 100, 2)}%)\n")


In [11]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas duplicadas completas de un DataFrame.

    Parametros:
        df (pd.DataFrame): DataFrame de entrada.

    Retorna:
        pd.DataFrame: DataFrame con las filas duplicadas.
    """
    duplicate_rows = df[df.duplicated(keep=False)]
    
    if duplicate_rows.empty:
        print("✅ No hay filas duplicadas en el dataset.")
    else:
        print(f"⚠ Hay {duplicate_rows.shape[0]} filas duplicadas en el dataset.")
        display(duplicate_rows)
    
    return duplicate_rows

In [12]:
def revisar_valores_nulos(df):
    """
    Muestra la cantidad y porcentaje de valores nulos en cada columna.

    Parametros:
        df (pd.DataFrame): DataFrame a analizar.

    Retorna:
        pd.DataFrame: DataFrame con informacion sobre valores nulos.
    """
    nulos = df.isnull().sum()
    porcentaje_nulos = round((nulos / len(df)) * 100, 2)
    no_nulos = df.notna().sum()
    porcentaje_no_nulos = round((no_nulos / len(df)) * 100, 2)
    
    df_nulos = pd.DataFrame({
        "Valores nulos": nulos, 
        "% Nulos": porcentaje_nulos.astype(str) + "%", # astype(str) para convertir float a str y poder concatenar con %
        "% No Nulos": porcentaje_no_nulos.astype(str) + "%",
        "Valores Unicos": df.nunique(),
        "Tipo de Dato": df.dtypes
    })

    print(f"📌 Columnas con valores nulos: {len(df_nulos[df_nulos['Valores nulos'] > 0])}")
    print(f"📌 Columnas sin valores nulos: {len(df_nulos[df_nulos['Valores nulos'] == 0])}")
    
    display(df_nulos[df_nulos["Valores nulos"] > 0])  # Muestra solo columnas con nulos
    
    return df_nulos

In [13]:
def obtener_estadisticas(df):
    """
    Muestra estadisticas descriptivas de variables numericas y categoricas.

    Parametros:
        df (pd.DataFrame): DataFrame a analizar.

    Retorna:
        None
    """
    print("📌 Estadisticas descriptivas de variables numericas:")
    display(df.describe().T)

    print("\n📌 Estadisticas descriptivas de variables categoricas:")
    display(df.describe(include="O").T)  # O = Object (cadenas de texto)

In [14]:
def revisar_valores_unicos(df):
    """
    Muestra los valores unicos y sus frecuencias en cada variable categorica.

    Parametros:
        df (pd.DataFrame): DataFrame a analizar.

    Retorna:
        None
    """
    columnas_categoricas = df.select_dtypes(include='object').columns.tolist()
    
    if not columnas_categoricas:
        print("✅ No hay columnas categoricas en el dataset.")
        return
    
    print("📌 Analisis de valores unicos en variables categoricas:\n")
    
    for columna in columnas_categoricas:
        print(f"\n----------- ANALIZANDO: '{columna.upper()}' -----------\n")
        print(f"Valores unicos: {df[columna].unique()}\n")
        print(f"Frecuencia de valores:\n{df[columna].value_counts()}\n")

## APLICACION FUNCIONES DE CARGA Y EXPLORACION ✨

In [16]:
# cargar datos
df = cargar_datos("../datos/bruto/Bank_Customer_Churn_Prediction.csv")

✅ Datos cargados correctamente. Filas: 10000, Columnas: 12


,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
# Revisar estructura del dataset
revisar_estructura(df)

📌 Dimensiones del dataset:
  - Filas: 10000
  - Columnas: 12

📌 Nombres de las columnas:
['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure', 'balance', 'products_number', 'credit_card', 'active_member', 'estimated_salary', 'churn'] 

📌 Tipos de datos:
customer_id           int64
credit_score          int64
country              object
gender               object
age                   int64
tenure                int64
balance             float64
products_number       int64
credit_card           int64
active_member         int64
estimated_salary    float64
churn                 int64
dtype: object 

📌 Datos duplicados: 0 (0.0%)



In [18]:
# Identificar filas duplicadas
get_duplicate_rows(df)

✅ No hay filas duplicadas en el dataset.


,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn


In [19]:
# Revisar valores nulos y duplicados
df_nulos = revisar_valores_nulos(df)

📌 Columnas con valores nulos: 0
📌 Columnas sin valores nulos: 12


,Valores nulos,% Nulos,% No Nulos,Valores Unicos,Tipo de Dato


In [20]:
# Obtener estadisticas descriptivas
obtener_estadisticas(df)

📌 Estadisticas descriptivas de variables numericas:


,count,mean,std,min,25%,50%,75%,max
customer_id,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
credit_score,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
products_number,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
credit_card,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
active_member,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
estimated_salary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48
churn,10000.0,2.037000e-01,0.402769,0.00,0.00,0.000000e+00,0.000000e+00,1.00



📌 Estadisticas descriptivas de variables categoricas:


,count,unique,top,freq
country,10000,3,France,5014
gender,10000,2,Male,5457


In [21]:
# Revisar valores unicos en variables categoricas
revisar_valores_unicos(df)

📌 Analisis de valores unicos en variables categoricas:


----------- ANALIZANDO: 'COUNTRY' -----------

Valores unicos: ['France' 'Spain' 'Germany']

Frecuencia de valores:
country
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64


----------- ANALIZANDO: 'GENDER' -----------

Valores unicos: ['Female' 'Male']

Frecuencia de valores:
gender
Male      5457
Female    4543
Name: count, dtype: int64

